# Multi-omics Enabled Sample Mislabeling Correction Challenge

This notebook is using various classifiers in an attempt to detect sample misclassifications

Details about this challenge: https://precision.fda.gov/challenges

## Solution

Import libraries

In [1]:
import os
import sys
import getopt
import re
import pandas as pd
from sklearn.model_selection import cross_val_score, GridSearchCV, cross_validate, train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

Load data

In [12]:
labels = pd.read_csv("challenge_data/train_cli.tsv", sep="\t", index_col="sample")
proteins = pd.read_csv("challenge_data/train_pro.tsv", sep="\t")
# Transpose proteins matrix
proteins = proteins.T
misClassified = pd.read_csv("challenge_data/sum_tab_1.csv", sep=",")
# Replace missing values with median
proteins = proteins.fillna(proteins.median())
# Drop remaining columns with missing values
proteins = proteins.dropna(axis='columns')

# What if missing values are not really missing values but missing genes (Y chrom for instance)
# proteins = proteins.fillna(0)

Select only rows which were correctly classified (matches) for machine learning

In [13]:
matches = list(misClassified.query('mismatch==0').loc[:,"sample"])
x = proteins.loc[matches]
y = labels.loc[matches]

## Initial Classifications
First exploration of how different classifiers perform

Function for combining the labels

In [14]:
def combineLabels(a, b):
    combined =  [None] * len(a)
    for i in range(len(a)):
        if (a[i] == 0 and b[i] == 0):
            combined[i] = 0
        if (a[i] == 0 and b[i] == 1):
            combined[i] = 1
        if (a[i] == 1 and b[i] == 0):
            combined[i] = 2
        if (a[i] == 1 and b[i] == 1):
            combined[i] = 3
    return combined

Train and test a classifier on msi, gender and combined labels

In [15]:
def classify(x, y, clf):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, shuffle=True)
    lb = preprocessing.LabelBinarizer()
    # I will have separat models for gende and msi
    y_gender_train = lb.fit_transform(y_train.loc[:,"gender"]).ravel()
    y_gender_test = lb.fit_transform(y_test.loc[:,"gender"]).ravel()
    y_msi_train = lb.fit_transform(y_train.loc[:,"msi"]).ravel()
    y_msi_test = lb.fit_transform(y_test.loc[:,"msi"]).ravel()

    y_combined_train = combineLabels(y_gender_train, y_msi_train)
    y_combined_test = combineLabels(y_gender_test, y_msi_test)

    clf.fit(x_train, y_gender_train)

    y_gender_predict = clf.predict(x_train)
    print("Gender train accuracy:", accuracy_score(y_gender_train, y_gender_predict))

    y_gender_predict = clf.predict(x_test)
    print("Gender test accuracy:", accuracy_score(y_gender_test, y_gender_predict))

    clf.fit(x_train, y_msi_train)

    y_msi_predict = clf.predict(x_train)
    print("Msi train accuracy:", accuracy_score(y_msi_train, y_msi_predict))

    y_msi_predict = clf.predict(x_test)
    print("Msi test accuracy:", accuracy_score(y_msi_test, y_msi_predict))
    
    clf.fit(x_train, y_combined_train)
    
    y_combined_predict = clf.predict(x_train)
    print("Combined train accuracy:", accuracy_score(y_combined_train, y_combined_predict))
    # print("Msi train F1:", f1_score(y_msi_train, y_msi_predict))

    y_combined_predict = clf.predict(x_test)
    print("Combined test accuracy:", accuracy_score(y_combined_test, y_combined_predict))
    # print("Msi train F1:", f1_score(y_msi_test, y_msi_predict))

### SVM

* It seems that a high penalty needs to be set for SVM, otherwise it assigns the more frequent label (female and low msi) to everything.

In [16]:
classify(x, y, SVC(C=100, kernel="rbf", gamma="scale", probability=True))

('Gender train accuracy:', 1.0)
('Gender test accuracy:', 0.7142857142857143)
('Msi train accuracy:', 1.0)
('Msi test accuracy:', 0.8571428571428571)
('Combined train accuracy:', 1.0)
('Combined test accuracy:', 0.5714285714285714)


### Random Forest

In [17]:
classify(x, y, RandomForestClassifier(n_estimators = 10))

('Gender train accuracy:', 0.9787234042553191)
('Gender test accuracy:', 0.6190476190476191)
('Msi train accuracy:', 1.0)
('Msi test accuracy:', 0.8095238095238095)
('Combined train accuracy:', 1.0)
('Combined test accuracy:', 0.42857142857142855)


### KNN

In [18]:
classify(x, y, KNeighborsClassifier(n_neighbors=4))

('Gender train accuracy:', 0.7872340425531915)
('Gender test accuracy:', 0.7142857142857143)
('Msi train accuracy:', 0.9361702127659575)
('Msi test accuracy:', 0.8095238095238095)
('Combined train accuracy:', 0.7659574468085106)
('Combined test accuracy:', 0.5238095238095238)


## Explore Parameters

Now, it's time to figure out the best parameters for each model

In [21]:
gender = y.loc[:,"gender"]
msi = y.loc[:,"msi"]

In [22]:
def bestParams(x, y, clf, grid):
    grid_search = GridSearchCV(clf, param_grid=grid, cv=10, iid=False)
    grid_search.fit(x, y)
    print("Tuned params:", grid_search.best_params_)
    print("Tuned best acc:", grid_search.best_score_)

### Random Forest

In [7]:
grid = {
    "n_estimators": range(5, 35, 5),
    "max_depth": range(4, 120, 5)
}
print("Best params for gender")
bestParams(x, gender, RandomForestClassifier(), grid)
print("Best params for msi")
bestParams(x, msi, RandomForestClassifier(), grid)

Best params for gender
('Tuned params:', {'n_estimators': 25, 'max_depth': 74})
('Tuned best acc:', 0.7410714285714286)
Best params for msi
('Tuned params:', {'n_estimators': 5, 'max_depth': 59})
('Tuned best acc:', 0.8875000000000002)


### SVM

In [119]:
grid = {
    "C": [pow(10,i) for i in range(-2,4)],
    "kernel": ["linear", "rbf"],
    "gamma": ["auto", "scale"]
}
print("Best params for gender")
bestParams(x, gender, SVC(), grid)
print("Best params for msi")
bestParams(x, msi, SVC(), grid)

Best params for gender
('Tuned params:', {'kernel': 'rbf', 'C': 10, 'gamma': 'auto'})
('Tuned best acc:', 0.6708333333333334)
Best params for msi
('Tuned params:', {'kernel': 'linear', 'C': 0.01, 'gamma': 'auto'})
('Tuned best acc:', 0.9)


### KNN

In [123]:
grid = {
    "n_neighbors": range(1,20)
}
print("Best params for gender")
bestParams(x, gender, KNeighborsClassifier(), grid)
print("Best params for msi")
bestParams(x, msi, KNeighborsClassifier(), grid)

Best params for gender
('Tuned params:', {'n_neighbors': 5})
('Tuned best acc:', 0.7345238095238096)
Best params for msi
('Tuned params:', {'n_neighbors': 9})
('Tuned best acc:', 0.9041666666666668)


### Logistic Regression

In [202]:
grid = {
    "solver": ["lbfgs", "liblinear"]
}
print("Best params for gender")
bestParams(x, gender, LogisticRegression(), grid)
print("Best params for msi")
bestParams(x, msi, LogisticRegression(), grid)


Best params for gender
('Tuned params:', {'solver': 'lbfgs'})
('Tuned best acc:', 0.6708333333333333)
Best params for msi
('Tuned params:', {'solver': 'lbfgs'})
('Tuned best acc:', 0.8875)


### ADA Boost

In [270]:
grid = {
    "n_estimators": range(40, 60, 5)
}
print("Best params for gender")
bestParams(x, gender, AdaBoostClassifier(), grid)
print("Best params for msi")
bestParams(x, msi, AdaBoostClassifier(), grid)

Best params for gender
('Tuned params:', {'n_estimators': 45})
('Tuned best acc:', 0.7761904761904762)
Best params for msi
('Tuned params:', {'n_estimators': 45})
('Tuned best acc:', 0.8416666666666668)


## How to Combine MSI and Gender?

Msi seems to be better indicator than gender. How do we take this into account?

* MSI does not match --> Mismatch label, no matter what gender says
* MSI matching, gender mismatch - what do we do?
* I propose to calculate a confidence score and use it in this case.


### Confidence Score
* Instead of reporting just label, show model's confidence score. This can help us decide in case of matching msi and gender misclassification.

In [51]:
clf = SVC(C=100, kernel="rbf", gamma="scale", probability=True)
#clf = RandomForestClassifier(n_estimators=100)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, shuffle=True, random_state=100)
lb = preprocessing.LabelBinarizer()
y_gender_train = lb.fit_transform(y_train.loc[:,"gender"]).ravel()
y_gender_test = lb.fit_transform(y_test.loc[:,"gender"]).ravel()
y_msi_train = lb.fit_transform(y_train.loc[:,"msi"]).ravel()
y_msi_test = lb.fit_transform(y_test.loc[:,"msi"]).ravel()

clf.fit(x_train, y_gender_train)
y_gender_predict = clf.predict(x_test)
print("Gender test accuracy:", accuracy_score(y_gender_test, y_gender_predict))
probs = clf.predict_proba(x_test)
for i in range(len(probs)):
    print(probs[i] , y_gender_test[i])
print()

clf.fit(x_train, y_msi_train)
y_msi_predict = clf.predict(x_test)
print("Msi test accuracy:", accuracy_score(y_msi_test, y_msi_predict))
probs = clf.predict_proba(x_test)
for i in range(len(probs)):
    print(probs[i], y_msi_test[i])

('Gender test accuracy:', 0.6190476190476191)
(array([0.68232265, 0.31767735]), 1)
(array([0.69272413, 0.30727587]), 0)
(array([0.68428927, 0.31571073]), 1)
(array([0.68667506, 0.31332494]), 0)
(array([0.6249667, 0.3750333]), 1)
(array([0.45015556, 0.54984444]), 1)
(array([0.82886351, 0.17113649]), 1)
(array([0.66656119, 0.33343881]), 0)
(array([0.66511439, 0.33488561]), 1)
(array([0.74174193, 0.25825807]), 0)
(array([0.78911414, 0.21088586]), 1)
(array([0.68384196, 0.31615804]), 0)
(array([0.81968649, 0.18031351]), 0)
(array([0.65743684, 0.34256316]), 0)
(array([0.74588987, 0.25411013]), 0)
(array([0.64746958, 0.35253042]), 0)
(array([0.86540917, 0.13459083]), 1)
(array([0.66728588, 0.33271412]), 1)
(array([0.76230683, 0.23769317]), 0)
(array([0.55475304, 0.44524696]), 1)
(array([0.75708745, 0.24291255]), 0)
()
('Msi test accuracy:', 1.0)
(array([0.15381273, 0.84618727]), 1)
(array([0.03480029, 0.96519971]), 1)
(array([0.1559887, 0.8440113]), 1)
(array([0.08524463, 0.91475537]), 1)
(a

Unfortunately, it does not look like the confidence is helpful when predicting gender

## Final Classification

Train all classifiers with best parameters (discovered by grid search) and run them on the test set.
Each time, use a different subset (80%) to give each model slightly different data to reduce overfitting

In [23]:
def finalPredict(x_train, y_train, x_test, y_test, clf):
    x_train_subset, a, y_train_subset, b = train_test_split(x_train, y_train, test_size=0.2, shuffle=True)
    clf.fit(x_train_subset, y_train_subset)
    y_predict = clf.predict(x_test)
    print("Test accuracy:", accuracy_score(y_test, y_predict))
    return y_predict

Predict with all

In [25]:
# Load test data in the same way train data was loaded
labels_test = pd.read_csv("challenge_data/test_cli.tsv", sep="\t", index_col="sample")
proteins_test = pd.read_csv("challenge_data/test_pro.tsv", sep="\t")
proteins_test = proteins_test.T
proteins_test = proteins_test.fillna(proteins.median())
proteins_test = proteins_test.dropna(axis='columns')

gender_test = labels_test.loc[:,"gender"]
msi_test = labels_test.loc[:,"msi"]

predictions =  [None] * 8

predictions[0] = finalPredict(x, gender, proteins_test, gender_test, RandomForestClassifier(n_estimators = 10, max_depth = 104))
predictions[4] = finalPredict(x, msi, proteins_test, msi_test, RandomForestClassifier(n_estimators = 10, max_depth = 39))

predictions[1] = finalPredict(x, gender, proteins_test, gender_test, SVC(C=100, kernel="rbf", gamma="auto"))
predictions[5] = finalPredict(x, msi, proteins_test, msi_test, SVC(C=0.01, kernel="linear", gamma="auto"))

predictions[2] = finalPredict(x, gender, proteins_test, gender_test, KNeighborsClassifier(n_neighbors=7))
predictions[6] = finalPredict(x, msi, proteins_test, msi_test, KNeighborsClassifier(n_neighbors=9))

# Results of logistic regression are too similar to SVM, there is no point in using it
#predictions[3] = finalPredict(x, gender, proteins_test, gender_test, LogisticRegression(solver="lbfgs"))
#predictions[7] = finalPredict(x, msi, proteins_test, msi_test, LogisticRegression(solver="lbfgs"))

predictions[3] = finalPredict(x, gender, proteins_test, gender_test, AdaBoostClassifier(n_estimators = 45))
predictions[7] = finalPredict(x, msi, proteins_test, msi_test, AdaBoostClassifier(n_estimators = 45))

('Test accuracy:', 0.45)
('Test accuracy:', 0.8125)
('Test accuracy:', 0.3375)
('Test accuracy:', 0.85)
('Test accuracy:', 0.375)
('Test accuracy:', 0.8625)
('Test accuracy:', 0.5)
('Test accuracy:', 0.7375)


If the predictions of all models agree and they are different from annotation, set the mislabeled flag to 1.

In [36]:
extensiveOut = open("extensive.csv","w+")
finalOut = open("submission.csv","w+")
names = list(labels_test.index)

extensiveOut.write("sample,Test_gender,Test_msi, RandomForest_gender,SVM_gender,KNN_gender,ADA_gender,RandomForest_msi,SVM_msi,KNN_msi,ADA_msi,gender_mislabeled,msi_mislabeld,mislabeled\n")
finalOut.write("sample,mismatch\n")
for i in range(len(predictions[0])):
    gender_mislabeled = "0"
    msi_mislabeled = "0"
    mislabeled = "0"
    if (predictions[0][i] == predictions[1][i] and predictions[0][i] == predictions[2][i] 
        and predictions[0][i] == predictions[3][i] and predictions[0][i] != labels_test.iloc[i,0]):
        gender_mislabeled = "1"
        
    if (predictions[4][i] == predictions[5][i] and predictions[4][i] == predictions[6][i] 
        and predictions[4][i] == predictions[7][i] and predictions[4][i] != labels_test.iloc[i,1]):
        msi_mislabeled = "1"
        
    if (msi_mislabeled == "1" or gender_mislabeled == "1"):
        mislabeled = "1"

    extensiveOut.write(names[i] + "," + labels_test.iloc[i,0] + "," + labels_test.iloc[i,1])
    for j in range(len(predictions)):
        extensiveOut.write("," + predictions[j][i])
    extensiveOut.write(", " + gender_mislabeled + "," + msi_mislabeled + "," + mislabeled)
    extensiveOut.write("\n")
    
    finalOut.write(names[i] + "," + mislabeled + "\n")

extensiveOut.close()
finalOut.close()
